## **Red Black Tree**

In [5]:
from time import time
from random import randint

In [6]:
class Node:
    def __init__(self,data,red=True):
        self.data = data
        self.left = None
        self.right = None
        self.parent = None
        self.red = red

In [7]:
class RedBlackTree:
    def __init__(self):
        self.root = Node(None,False)
        self.root.left = Node(None,False)
        self.root.right = Node(None,False)

    def insert(self,data):
        node = self.newNode(data)

        root = self.root
        temp = None

        while root.data is not None:
            temp = root
            if node.data < root.data:
                root = root.left
            else:
                root = root.right
        
        node.parent = temp
        if temp is None:
            self.root = node
        elif node.data < temp.data:
            temp.left = node
        else:
            temp.right = node
        
        if node.parent is None:
            node.red = False
            return

        if node.parent.parent is None:
            return

        self.fixInsertion(node)
        

    def fixInsertion(self,node):
        while node.parent.red:
            parent = node.parent
            grandparent = node.parent.parent
            if grandparent.left == parent:
                uncle = grandparent.right
                # recolor
                if uncle.red:
                    parent.red = False
                    uncle.red = False
                    grandparent.red = True
                    node = grandparent
                # if triangle, create line
                else:
                    if node == parent.right:
                        node = node.parent
                        self.leftRotate(node)
                    node.parent.red = False
                    node.parent.parent.red = True
                    self.rightRotate(node.parent.parent)
            
            else:
                uncle = grandparent.left
                if uncle.red:
                    parent.red = False
                    uncle.red = False
                    grandparent.red = True
                    node = grandparent
                else:
                    if node == parent.left:
                        node = node.parent
                        self.rightRotate(node)
                    node.parent.color = False
                    node.parent.parent.color = True
                    self.leftRotate(node.parent.parent)

            if node == self.root or node.parent == self.root:
                break

        self.root.red = False
        

        ### Recursion Version
        # if node == self.root:
        #     node.red = False
        #     return node
        
        # parent = node.parent
        # if parent.red:
        #     # case 0: make root black
        #     if parent == self.root:
        #         parent.red = False
        #         return node
        
        #     grandparent = parent.parent
        #     uncle = None

        #     if parent == grandparent.left:
        #         uncle = grandparent.right
        #         # case 1: recolor
        #         if uncle.red:
        #             print("uncle red left")
        #             parent.red = False
        #             uncle.red = False
        #             grandparent.red = True
        #             self.fixInsertion(grandparent)
        #         # case 2: triangle
        #         elif parent.right == node:
        #             print("triangle left")
        #             self.leftRotate(parent)
        #             self.fixInsertion(grandparent.left.left)
        #         # line
        #         else:
        #             print("line left")
        #             parent.red = False
        #             grandparent.red = True
        #             self.rightRotate(grandparent)

        #     else:
        #         uncle = grandparent.left
        #         if uncle.red:
        #             print("uncle red right")
        #             parent.red = False
        #             uncle.red = False
        #             grandparent.red = True
        #             self.fixInsertion(grandparent)
                
        #         elif parent.left == node:
        #             print("triangle right")
        #             self.rightRotate(parent)
        #             self.fixInsertion(grandparent.right.right)
                
        #         else:
        #             print("line right")
        #             parent.red = False
        #             grandparent.red = True
        #             self.leftRotate(grandparent)
        # return node



    def leftRotate(self, root):
        y = root.right
        root.right = y.left

        if y.left.data is not None:
            y.left.parent = root
        
        y.parent = root.parent

        if root.parent is None:
            self.root = y
        elif root == root.parent.left:
            root.parent.left = y
        else:
            root.parent.right = y
        y.left = root
        root.parent = y

    def rightRotate(self, root):
        y = root.left
        root.left = y.right
        if y.right.data is not None:
            y.right.parent = root

        y.parent = root.parent

        if root.parent == None:
            self.root = y
        elif root == root.parent.right:
            root.parent.right = y
        else:
            root.parent.left = y
            
        y.right = root
        root.parent = y
    
    # new node has 2 black nil children
    def newNode(self, data):
        node = Node(data,True)
        node.left = Node(None,False)
        node.right = Node(None,False)
        return node

    def inOrder(self, root):
        if not root or not root.data:
            return
        self.inOrder(root.left)
        print(root.data, root.red)
        self.inOrder(root.right)
        
        

Insertion Test

In [19]:
N = 10000


for test in range(5):

    rb_tree = RedBlackTree()

    start = time()
    for i in range(1,N):
        rb_tree.insert(randint(1,1000))
    end = time()

    print(f"{end-start:.8f}")

# rb_tree.inOrder(rb_tree.root)

0.08821702
0.02989483
0.03241324
0.03488493
0.07029843
